In [1]:
import SNAPy as sna
import geopandas as gpd
import madina as md
import madina.una.tools as una
import numpy as np
from IPython.display import display
import ipywidgets as widgets


In [2]:
# MainDir = 'D:\\OneDrive\\0203 GIS Files\\0110 Jakarta\\'
# dfNetwork = gpd.read_file(MainDir+'Jakpus_sample_sgt.gpkg', layer='edges').set_crs(32748, allow_override=True) # network dataframe
# dfEntries= gpd.read_file(MainDir+'Jakpus_sample.gpkg', layer='entries') # entrance dataframe

In [3]:
MainDir = 'D:\\OneDrive\\0203 GIS Files\\0110 Jakarta\\'
dfNetwork = gpd.read_file(MainDir+'testSegment.gpkg', layer='edges').set_crs(32748, allow_override=True) # network dataframe
dfEntries= gpd.read_file(MainDir+'Jakpus_sample.gpkg', layer='entries') # entrance dataframe

In [4]:
baseSet = {
        'EntDist' : 100,
        'EntID': 'fid',
        'EdgeID': 'fid',
        'Threads': 0,
    }

nwSim = sna.GraphSims(dfNetwork, dfEntries, **baseSet) # Restarting nwSim

GraphSim Class ----------
Projection EPSG:32748
EntriesDf EntID not detected, adding from index
NetworkDf EdgeID not detected, adding from index
Graph Built with 56,585 Nodes, 74,311 Edges
Graph mapped 14,039 Entries
Graph initialization finished in 7.0353 s


In [ ]:
import time
a = widgets.FloatSlider(
    value=7.5,
    min=200,
    max=2000.0,
    step=0.1,
    description='Distance:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    width='500px',
)

EntID = [1200,]

rslt = nwSim.PathReach(EntID, a.value, True, True, True, True)
ddeck = rslt[0]

output = widgets.Output()

def valcg(change):
    with output:
        output.clear_output(wait=True)
        rsltB = nwSim.PathReach(EntID, a.value, True, True, True, True, True)
        ddeck.layers = rsltB
        display(ddeck)
with output:
    display(ddeck)
a.observe(valcg)
display(a, output)

In [ ]:
rslt = nwSim.PathReach([3000,], 785, True, True, True, True)
rslt[0].to_html('spread.html')

In [ ]:
from shapely.geometry import LineString, MultiLineString, Point, mapping, shape
from shapely.ops import nearest_points, split, linemerge

eid = [x[0] for x in rslt]
edf = nwSim.NetworkDf.loc[eid][['geometry', 'fid']]
edf['vector'] = [x[1] for x in rslt]



edf['geometry'] = edf.apply(lambda x: splitline(x.geometry, x.vector), axis=1)


In [ ]:
edf.reset_index()

In [ ]:
nwSim.Map_BaseLayerInit()

In [ ]:
nwSim.Map_PopLayer()

In [ ]:
nwSim.Map_Show()

In [ ]:
baseSet = {
    'DetourR' : 1.1, 
    'SearchDist' : 400.0,
    'OriWgt': 'Kapasitas',
    'DestWgt': 'Kapasitas',
    'RsltAttr': 'Btwn',
    'Threads':0,
    'Include_Destination':True
}
nwSim.EntriesDf['Kapasitas'] = 100.0
idO = nwSim.EntriesDf[nwSim.EntriesDf['jenbang'].isin(('Hunian', 'HUNIAN'))].fid
idD = nwSim.EntriesDf[nwSim.EntriesDf['jenbang'].isin(('Non Hunian',))].fid
nwSim.BetweenessPatronage(idO, idD, **baseSet)

In [ ]:
nwSim.Map_BaseLayerInit()
nwSim.Map_LayerAdd(
    {'Btwn':{'Omin':8.0, 'Omax':15.0}}
)
a = nwSim.Map_Show()
a

In [18]:
nwSim.NetworkDf.to_file(MainDir+"testRun.gpkg", layer="network", driver='GPKG')

In [ ]:
nwSim.Map_LayerAdd(
    {'Btwn_M03': None,}
)
nwSim.Map_Show()

In [ ]:
dfEntries['Kapasitas'] = 100.0
mdSim = md.Zonal()
mdSim.load_layer(name='sidewalks', source=dfNetwork)
mdSim.load_layer(name='entOrig', source=dfEntries)
mdSim.load_layer(name='entDest', source=dfEntries)
mdSim['entOrig'].gdf = mdSim['entOrig'].gdf[mdSim['entOrig'].gdf['jenbang'].isin(('Hunian', 'HUNIAN'))]
mdSim['entDest'].gdf = mdSim['entDest'].gdf[mdSim['entDest'].gdf['jenbang'].isin(('Non Hunian',))]


mdSim.create_street_network(source_layer='sidewalks', node_snapping_tolerance=0.1)
mdSim.insert_node(label='origin', layer_name='entOrig', weight_attribute='Kapasitas')
mdSim.insert_node(label='destination', layer_name='entDest', weight_attribute='Kapasitas')
mdSim.create_graph()
mdSim.describe()

In [ ]:
una.betweenness(
    mdSim,
    search_radius = 800,
    save_betweenness_as="baseline",
    detour_ratio=1.1,
    num_cores=16
)


In [ ]:
sets = [
    ['Btwn_M04', 8.0, 1, 0.8],
    ['Btwn_M03', 12.0, 2.5, 1.1],
]
rslt = sna.SimTimeDistribute(nwSim.NetworkDf, sets, spread=0.5, cal_ends=False)
rslt

In [ ]:
nwSim.NetworkDf.to_file(MainDir+"testresults.gpkg", layer="NetworkQlt", driver='GPKG')